In [1]:
import tensorflow as tf

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.5.0
Eager execution: True


In [2]:
def load(path: str) -> tf.data.TFRecordDataset:
    raw_dataset = tf.data.TFRecordDataset(path)
    # 特徴の記述
    feature_description = {
        'petal_length': tf.io.FixedLenFeature([], tf.float32, default_value=0),
        'petal_width': tf.io.FixedLenFeature([], tf.float32, default_value=0),
        'target': tf.io.FixedLenFeature([3], tf.float32, default_value=[0, 0, 0]),
    }

    def _parse_function(example_proto):
        return tf.io.parse_single_example(example_proto, feature_description)
    
    def _feature_convert_function(feature):
#         target = feature.pop("target")
        return [feature["petal_length"], feature["petal_width"]], feature["target"]
    
    parsed_dataset = (
        raw_dataset
        .map(_parse_function)
        .map(_feature_convert_function)
    )
    return parsed_dataset


train_dataset = load('../data/02_features/iris/train.tfrecord')
test_dataset = load('../data/02_features/iris/test.tfrecord')

for record in train_dataset.take(5):
    print(record)

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.6566521, 1.0723119], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 1.], dtype=float32)>)
(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.8289515, 1.0723119], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 1.], dtype=float32)>)
(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-1.4109405, -1.1992522], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 0., 0.], dtype=float32)>)
(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.36948645, 0.00334054], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 0.], dtype=float32)>)
(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.08997857,  0.13696197], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 0.], dtype=float32)>)


In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(1, 1)),  # input shape required
    tf.keras.layers.Dense(10, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.relu),
    tf.keras.layers.Dense(3, activation=tf.nn.softmax),
])

model.compile(    
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

model.fit(
    train_dataset,
    epochs=10,
#     validation_data=validation_dataset,
#     callbacks=callbacks,
)
print(model.evaluate(test_dataset))
model.save(filepath="../data/03_models/iris.h5", save_format="h5")


Epoch 1/10
120/120 [==============================] - 0s 823us/step - loss: 0.2231 - root_mean_squared_error: 0.4723
Epoch 2/10
120/120 [==============================] - 0s 754us/step - loss: 0.2222 - root_mean_squared_error: 0.4714
Epoch 3/10
120/120 [==============================] - 0s 706us/step - loss: 0.2222 - root_mean_squared_error: 0.4714
Epoch 4/10
120/120 [==============================] - 0s 780us/step - loss: 0.2222 - root_mean_squared_error: 0.4714
Epoch 5/10
120/120 [==============================] - 0s 945us/step - loss: 0.2222 - root_mean_squared_error: 0.4714
Epoch 6/10
120/120 [==============================] - 0s 736us/step - loss: 0.2222 - root_mean_squared_error: 0.4714
Epoch 7/10
120/120 [==============================] - 0s 1ms/step - loss: 0.2222 - root_mean_squared_error: 0.4714
Epoch 8/10
120/120 [==============================] - 0s 728us/step - loss: 0.2222 - root_mean_squared_error: 0.4714
Epoch 9/10
120/120 [==============================] - 0s 754us/ste